In [1]:
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm, trange

In [2]:
url = "https://www.lamaistas.lt/visi-receptai/"
output_file = "urls.txt"

In [3]:
n = round(25419/44)
def Scrape(url, output_file):
    headers = {"User-Agent": "Mozilla/5.0"}

    for i in tqdm(range(1, n+1), desc="Scraping pages"):
        new_url = f"{url}{i}"
        try:
            response = requests.get(new_url, headers=headers)
            soup = BeautifulSoup(response.text, "html.parser")

            for a in soup.find_all("a", href=True):
                if "/receptas/" in a["href"]:
                    full_link = a["href"]
                    with open(output_file, "a", encoding="utf-8") as f:
                        f.write(full_link + "\n")
            time.sleep(0.3)
        except:
            print(f"Error in page {i}")
            continue

In [4]:
Scrape(url, output_file)

Scraping pages: 100%|██████████| 578/578 [07:26<00:00,  1.29it/s]


In [6]:
#Delete duplicates
with open(output_file, "r", encoding="utf-8") as f:
    lines = f.readlines()
    unique_lines = list(set(lines))

with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(unique_lines)